# Transfer Learning and Fine Tuning

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

#### Using GPU (highly recommended)

-> If using `theano` backend:

`THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32`

In [1]:
import numpy as np
import datetime

np.random.seed(1337)  # for reproducibility

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from numpy import nan
import keras

print keras.__version__
now = datetime.datetime.now

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1050 Ti (CNMeM is enabled with initial size: 3.0% of memory, cuDNN 5105)


2.0.8


### Settings

In [3]:
now = datetime.datetime.now

batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [5]:
def train_model(model, train, test, nb_classes):
    
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(X_train, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

## Dataset Preparation

In [6]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

In [7]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  border_mode='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

/home/user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid", input_shape=(28, 28, 1...)`
/home/user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`


In [8]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (X_train_lt5, y_train_lt5),
            (X_test_lt5, y_test_lt5), nb_classes)

('X_train shape:', (30596, 28, 28, 1))
(30596, 'train samples')
(5139, 'test samples')


/home/user/anaconda2/lib/python2.7/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 6s - loss: 0.2184 - acc: 0.9342 - val_loss: 0.0541 - val_acc: 0.9815
Epoch 2/5
30596/30596 [==============================] - 6s - loss: 0.0745 - acc: 0.9767 - val_loss: 0.0274 - val_acc: 0.9905
Epoch 3/5
30596/30596 [==============================] - 6s - loss: 0.0488 - acc: 0.9852 - val_loss: 0.0187 - val_acc: 0.9946
Epoch 4/5
30596/30596 [==============================] - 6s - loss: 0.0375 - acc: 0.9889 - val_loss: 0.0142 - val_acc: 0.9951
Epoch 5/5
30596/30596 [==============================] - 6s - loss: 0.0339 - acc: 0.9898 - val_loss: 0.0124 - val_acc: 0.9957
Training time: 0:03:34.903606
('Test score:', 0.012447581390064992)
('Test accuracy:', 0.9957190114808329)


In [9]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (X_train_gte5, y_train_gte5),
            (X_test_gte5, y_test_gte5), nb_classes)

('X_train shape:', (29404, 28, 28, 1))
(29404, 'train samples')
(4861, 'test samples')
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 0s - loss: 0.3808 - acc: 0.8850 - val_loss: 0.0913 - val_acc: 0.9708
Epoch 2/5
29404/29404 [==============================] - 0s - loss: 0.1313 - acc: 0.9597 - val_loss: 0.0606 - val_acc: 0.9809
Epoch 3/5
29404/29404 [==============================] - 0s - loss: 0.0990 - acc: 0.9694 - val_loss: 0.0474 - val_acc: 0.9835
Epoch 4/5
29404/29404 [==============================] - 0s - loss: 0.0818 - acc: 0.9748 - val_loss: 0.0424 - val_acc: 0.9858
Epoch 5/5
29404/29404 [==============================] - 0s - loss: 0.0741 - acc: 0.9773 - val_loss: 0.0372 - val_acc: 0.9877
Training time: 0:00:07.915863
('Test score:', 0.03715517895759483)
('Test accuracy:', 0.9876568606178889)


## Your Turn

Try to Fine Tune a VGG16 Network

In [54]:
from keras.applications import VGG16
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
inp = Input(shape=(48,48,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(inp)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(5, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=inp, output=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

/home/user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=flatten/fl...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 48, 48, 3)         0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              2101248   
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 5)                 20485     
Total params: 33,617,733
Trainable params: 33,617,733
Non-trainable params: 0
________________________________________________________________

```python
...
...
# Plugging new Layers
    model.add(Dense(768, activation='sigmoid'))
    model.add(Dropout(0.0))
    model.add(Dense(768, activation='sigmoid'))
    model.add(Dropout(0.0))
    model.add(Dense(n_labels, activation='softmax'))
```

In [60]:
import scipy
new_shape = (48,48)
X_train_new = np.empty(shape=(X_train_gte5.shape[0],)+(48,48,3))

for idx in xrange(X_train_gte5.shape[0]):
    X_train_new[idx] = np.resize(scipy.misc.imresize(X_train_gte5[idx], (new_shape)), (48, 48, 3))
    X_train_new[idx] = np.resize(X_train_new[idx], (48, 48, 3))
    
#X_train_new = np.expand_dims(X_train_new, axis=-1)

print X_train_new.shape

X_train_new = X_train_new.astype('float32')
X_train_new /= 255
    
print('X_train shape:', X_train_new.shape)
print(X_train_new.shape[0], 'train samples')
print(X_train_new.shape[0], 'test samples')


# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train_gte5, nb_classes)
Y_test = np_utils.to_categorical(y_test_gte5, nb_classes)

print y_train.shape

my_model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

my_model.fit(X_train_new, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1)

#print('Training time: %s' % (now() - t))
#score = my_model.evaluate(X_test, Y_test, verbose=0)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])
    
    
    
#train_model(my_model,
#            (X_train_new, y_train_gte5),
#            (X_test_gte5, y_test_gte5), nb_classes)

/home/user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(29404, 48, 48, 3)
('X_train shape:', (29404, 48, 48, 3))
(29404, 'train samples')
(29404, 'test samples')
(60000,)


/home/user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:33: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/5
29404/29404 [==============================] - 203s - loss: 1.6217 - acc: 0.2112   
Epoch 2/5
29404/29404 [==============================] - 204s - loss: 1.6086 - acc: 0.2131   
Epoch 3/5
29404/29404 [==============================] - 204s - loss: 1.6086 - acc: 0.2131   
Epoch 4/5
29404/29404 [==============================] - 204s - loss: 1.6085 - acc: 0.2131   
Epoch 5/5
29404/29404 [==============================] - 204s - loss: 1.6085 - acc: 0.2131   
